In [1]:
import numpy as np
import qiskit.circuit.library as lib
from qiskit.quantum_info import Operator
from numpy.linalg import multi_dot
from IPython.display import display, Latex

def numpy_to_latex(matrix):
    latex_code = "\\begin{bmatrix}\n"
    for row in matrix:
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
    latex_code += "\\end{bmatrix}"
    return latex_code


Z = Operator(lib.ZGate())
X = Operator(lib.XGate())

def Ry(theta):
    return Operator(lib.RYGate(theta))

def P(theta):
    return Operator(lib.PhaseGate(theta))

u1 = multi_dot([X, Ry(np.arctan(1 + np.sqrt(3))), Z, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16)])
u2 = u1
v1 = multi_dot([X, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16), X, Ry(np.arctan(1 + np.sqrt(3))), Z, X])
v2 = multi_dot([X, Z, X, P(np.pi * 9 / 16), X, P(np.pi * 13 / 16), X, Z, Ry(np.arctan(1 + np.sqrt(3)))])
pi = np.pi
sqrt = np.sqrt
log = np.log

common_log_term = log(-1 / 6 * (-1) ** (1 / 4) * (sqrt(24 - 6 * sqrt(3)) + 1j * (3 + sqrt(3))))
common_factor = 3j + 1j * sqrt(3) + sqrt(6 * (4 - sqrt(3)))
denominator = 15 * pi - 4 * 1j * common_log_term

VCx = np.array([
    [
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator),
        0,
        0,
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator)
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator),
        0,
        0,
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator)
    ]
], dtype=complex)


fCheck = multi_dot([np.kron(u1, u2), VCx, np.kron(v1, v2)]).round(7)

zero_qubit = np.array([[1],[0]])
one_qubit = np.array([[0],[1]])
qubity = [zero_qubit, one_qubit]

In [80]:
fCheck

array([[ 0.5773503-0.j ,  0.5773503-0.j ,  0.5773503-0.j ,
        -0.       -0.j ],
       [ 0.5773503-0.j , -0.2886751+0.5j, -0.2886751-0.5j,
         0.       -0.j ],
       [ 0.5773503-0.j , -0.2886751-0.5j, -0.2886751+0.5j,
         0.       -0.j ],
       [-0.       -0.j ,  0.       -0.j ,  0.       -0.j ,
         1.       -0.j ]])

In [2]:
bellstatePhiMin = 1/np.sqrt(2) * (np.kron(qubity[0], qubity[1]) - np.kron(qubity[1], qubity[0]))
bellstatePhiPlus = 1/np.sqrt(2) * (np.kron(qubity[0], qubity[1]) + np.kron(qubity[1], qubity[0]))

In [81]:
wektorwlasny0 = np.array([ 0.888+0.j, -0.46 -0.j,  0.   -0.j])
wektorwlasny1 = np.array([ 0.325+0.j,  0.628+0.j, -0.707+0.j])
wektorwlasny2 = np.array([ 0.325+0.j,  0.628+0.j, 0.707+0.j])

In [82]:
wektorwlasny0

array([ 0.888+0.j, -0.46 +0.j,  0.   +0.j])

In [83]:
bellstatePhiMin

array([[ 0.        ],
       [ 0.70710678],
       [-0.70710678],
       [ 0.        ]])

In [84]:
bellstatePhiPlus

array([[0.        ],
       [0.70710678],
       [0.70710678],
       [0.        ]])

In [85]:
baza = [np.kron(qubity[1], qubity[1]), wektorwlasny0, wektorwlasny1]

In [86]:
stan0x0 = np.outer(baza[0], baza[0])
stan1x1 = np.outer(baza[1], baza[1])
stan2x2 = np.outer(baza[2], baza[2])

stanPlus = multi_dot([fCheck, baza[0]])

stanPrzygotowany = np.kron(fCheck, np.kron(np.identity(4), np.identity(4)))

Cz01 = np.kron(stan0x0, fCheck) + np.kron(stan1x1, multi_dot([fCheck, fCheck])) + np.kron(stan2x2, np.identity(4))
CZ01 = np.kron(Cz01, np.identity(4))



Cz02 = np.kron(np.kron(stan0x0, np.identity(4)), fCheck) + np.kron(np.kron(stan1x1, np.identity(4)), np.identity(4)) + np.kron(np.kron(stan2x2, np.identity(4)), multi_dot([fCheck, fCheck]))


#print(f'Cz02 = {Cz02}')

Cz12 = np.kron(np.identity(4), np.kron(stan0x0, np.identity(4)) + np.kron(stan1x1, fCheck) + np.kron(stan2x2, multi_dot([fCheck, fCheck])))



U = multi_dot([CZ01, Cz02, stanPrzygotowany])

#print(f'GHZ = {GHZ}')

stanStart = np.kron(baza[0], np.kron(baza[0], baza[0]))

GHZ = multi_dot([U, stanStart])

wspolczynnik_zmiany_bazy = np.kron(np.identity(4), np.kron(fCheck.conj().T, fCheck.conj().T))

GHZqubity = multi_dot([wspolczynnik_zmiany_bazy, GHZ])


ValueError: operands could not be broadcast together with shapes (16,16) (12,12) 

In [67]:
np.kron(np.outer(qubity[0], qubity[0]), np.identity(2)) + np.kron(np.outer(qubity[1], qubity[1]), Z)

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [68]:
stanPrzygotowany.shape

(64, 64)

In [69]:
GHZ.shape

(64, 1)

In [70]:
stan0x0

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]])

In [71]:
stan1x1

array([[ 0.10569001+0.j,  0.2041628 +0.j, -0.22987821+0.j,
         0.        +0.j],
       [ 0.2041628 +0.j,  0.394384  +0.j, -0.4440588 +0.j,
         0.        +0.j],
       [-0.22987821+0.j, -0.4440588 +0.j,  0.49999041-0.j,
        -0.        +0.j],
       [ 0.        +0.j,  0.        +0.j, -0.        +0.j,
         0.        +0.j]])

In [72]:
stan2x2

array([[0.10569001+0.j, 0.2041628 +0.j, 0.22987821+0.j, 0.        +0.j],
       [0.2041628 +0.j, 0.394384  +0.j, 0.4440588 +0.j, 0.        +0.j],
       [0.22987821+0.j, 0.4440588 +0.j, 0.49999041+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j]])

In [73]:
(multi_dot([fCheck, baza[1]]) * np.sqrt(3)).round(3)

array([0.246+0.j   , 0.365+1.156j, 0.365-1.156j, 0.   +0.j   ])

In [74]:
(multi_dot([fCheck, baza[2]]))

array([ 0.95851697+0.j     , -0.19771354-0.03955j, -0.19771354+0.03955j,
        0.        +0.j     ])

In [75]:
Cz01.round(3)

array([[ 0.211+0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.408+0.j ,
         0.   +0.j ,  0.   +0.j ,  0.   +0.j , -0.   +0.j , -0.   +0.j ,
        -0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,
         0.   +0.j ],
       [ 0.   +0.j ,  0.106+0.j ,  0.106+0.j ,  0.   +0.j ,  0.   +0.j ,
         0.204+0.j ,  0.204+0.j ,  0.   +0.j , -0.   +0.j ,  0.23 +0.j ,
        -0.23 +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,
         0.   +0.j ],
       [ 0.   +0.j ,  0.106+0.j ,  0.106+0.j ,  0.   +0.j ,  0.   +0.j ,
         0.204+0.j ,  0.204+0.j ,  0.   +0.j , -0.   +0.j , -0.23 +0.j ,
         0.23 +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,
         0.   +0.j ],
       [ 0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.211+0.j ,  0.   +0.j ,
         0.   +0.j ,  0.   +0.j ,  0.408+0.j ,  0.   +0.j ,  0.   +0.j ,
         0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,  0.   +0.j ,
         0.   +0.j ],
       [ 0.408+0.j ,  0.   +0.j ,  0

In [87]:
np.dot(Cz01, Cz01.conj().T).round(2)

array([[ 0.21+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.41+0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j, -0.  +0.j, -0.  +0.j, -0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.21+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.41+0.j,
         0.  +0.j,  0.  +0.j, -0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.21+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,
         0.41+0.j,  0.  +0.j, -0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j,  0.  +0.j,  0.21+0.j,  0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.41+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j],
       [ 0.41+0.j,  0.  +0.j,  0.  +0.j,  0.  +0.j,  0.79+0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j, -0.  +0.j, -0.  +0.j, -0.  +0.j,  0.  +0.j,
         0.  +0.j,  0.  +0.j,  0.  +0.j,  0.  +0

In [88]:
(GHZ * np.sqrt(3)).round(3)

array([[0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],


In [91]:
(GHZqubity).round(3)

array([[0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j

In [90]:
np.kron(baza[0], np.kron(baza[0], baza[0])) + np.kron(baza[1], np.kron(baza[1], baza[1])) + np.kron(baza[2], np.kron(baza[2], baza[2]))

array([[ 0.7345552 +0.j, -0.29639774+0.j, -0.07467688+0.j, ...,
         0.16245092+0.j,  0.31390517+0.j, -0.35339324+0.j],
       [ 0.7345552 +0.j, -0.29639774+0.j, -0.07467688+0.j, ...,
         0.16245092+0.j,  0.31390517+0.j, -0.35339324+0.j],
       [ 0.7345552 +0.j, -0.29639774+0.j, -0.07467688+0.j, ...,
         0.16245092+0.j,  0.31390517+0.j, -0.35339324+0.j],
       ...,
       [ 0.7345552 +0.j, -0.29639774+0.j, -0.07467688+0.j, ...,
         0.16245092+0.j,  0.31390517+0.j, -0.35339324+0.j],
       [ 0.7345552 +0.j, -0.29639774+0.j, -0.07467688+0.j, ...,
         0.16245092+0.j,  0.31390517+0.j, -0.35339324+0.j],
       [ 1.7345552 +0.j,  0.70360226+0.j,  0.92532313+0.j, ...,
         1.16245092+0.j,  1.31390517+0.j,  0.64660676+0.j]])

In [20]:
zeroQ3 = np.array([[1], [0], [0]])
oneQ3 = np.array([[0], [1], [0]])
twoQ3 = np.array([[0], [0], [1]])

In [33]:
zeroXzeroQ3 = np.outer(zeroQ3, zeroQ3)
oneXoneQ3 = np.outer(oneQ3, oneQ3)
twoXtwoQ3 = np.outer(twoQ3, twoQ3)

In [27]:
1j

1j

In [30]:
F = np.array([[1, 0, 0], [0, np.exp(np.pi*2/3 * 1j), 0], [0, 0, np.exp(np.pi*4/3 * 1j)]])

In [32]:
F.round(3)

array([[ 1. +0.j   ,  0. +0.j   ,  0. +0.j   ],
       [ 0. +0.j   , -0.5+0.866j,  0. +0.j   ],
       [ 0. +0.j   ,  0. +0.j   , -0.5-0.866j]])

In [35]:
czTEST = np.kron(zeroXzeroQ3, np.identity(3)) + np.kron(oneXoneQ3, F) + np.kron(twoXtwoQ3, multi_dot([F, F]))

In [36]:
czTEST

array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  1. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       , -0.5+0.8660254j,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       , -0.5-0.86

In [37]:
np.dot(czTEST, czTEST.conj().T)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        1.+0.j]])